## ABC Simple Installation System: Combine v01 and v02 results
(savings relative to thermal loads) \
Created on: 10/12/2021 \
By: Lixi Liu (Lixi.Liu@nrel.gov)

Note: change kernel to that of EULP Calibration & Validation before running

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
print(f'Notebook path: {os.getcwd()}')
from pathlib import Path
import eulpcv.resstock_enduse_categories as res_enduse_categories

### Download results online
* unprocessed upgrade results: S3/resbldg-datasets/abc-sis

### Initialize

In [ ]:
# output directory
pkg_version = f'packages' # <----- 
result_dir = Path('/Users/lliu2/Documents/ABC Simple Intall Sys/results')
resdir = result_dir / pkg_version / 'processed results'

## create folder for post-processed results:
outdir = result_dir / pkg_version / 'result summary'
plot_path = outdir / 'plots'

os.makedirs(resdir, exist_ok=True)
os.makedirs(outdir, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)

print(f'Output directory: \n   {outdir}')

### Variables

In [ ]:
# define what packages and upgrades to combine
packages_to_combine = ['packages_v01', 'packages_v02'] # <---

upgrades_by_packages_to_combine = {
    'packages_v01': [1,2,3,4,5,9,10],
    'packages_v02': [1,2,3,4,5]
}

upgrade_number_dict = {
    "Air Tight Envelope": 1, #(v01)
    "Ultra Air Tight Envelope - Mec Vent": 2, #(v01)
    "Ultra Air Tight Envelope - Rec Vent": 3, #(v01)
    "Air Tight Envelope + Roof Insulation": 4, #(v01)
    "Ultra Air Tight Envelope - Mech Vent + Roof Insulation": 5, #(V01)
    "Air Tight Envelope + MSHP": 6, #(v02, #1)
    "Ultra Air Tight Envelope - Mech Vent + MSHP": 7, #(v02, #2)
    "Ultra Air Tight Envelope - Rec Vent + MSHP": 8, #(v02, #3)
    "Air Tight Envelope + HPWH": 9, #(v01)
    "Ultra Air Tight Envelope - Mech Vent + HPWH": 10, #(v01)
    "Air Tight Envelope + MSHP _HPWH": 11, #(v02, #4)
    "Ultra Air Tight Envelope - Rec Vent + MSHP + HPWH": 12, #(v02, #5)    
}

print('combination process setting defined')

## Baseline result

In [ ]:
pkg = packages_to_combine[0]
resdir_path = result_dir / pkg / 'processed results'

res = pd.read_csv(resdir_path / 'results_up00.csv')

res.to_csv(resdir / 'results_up00.csv', index=False)
print('baseline result saved to new loc')

## Combine packages and rename ###

In [ ]:
## get summary table
summary_upgrades = []

for pkg in packages_to_combine:
    package_list = upgrades_by_packages_to_combine[pkg]
    print(f'\n>>> retrieving packages: {package_list} from {pkg}...')
    resdir_path = result_dir / pkg / 'processed results'

    for n in package_list:
        p = pd.read_csv(resdir_path / f'results_up{n:02d}.csv')
        print(f'  {pkg} - Package {n}')

        upgrade_name = p['apply_upgrade.upgrade_name'].replace('',np.nan).dropna(axis=0).unique()[0]
        upgrade_name_corrected = p['upgrade_name'].replace('',np.nan).dropna(axis=0).unique()[0]
        upgrade_name_short = p['upgrade_name_short'].replace('',np.nan).dropna(axis=0).unique()[0]
        package_no = upgrade_number_dict[upgrade_name]
        p['package_no'] = package_no

        # save processed results
        p.to_csv(resdir / f'results_up{package_no:02d}.csv', index=False)

        ### add to summary table
        summ = pd.DataFrame(index=[0])
        level0 = ''
        summ[(level0, 'upgrade_no')] = package_no
        summ[(level0, 'upgrade_name')] = upgrade_name_corrected
        summ[(level0, 'upgrade_name_short')] = upgrade_name_short
        summ[(level0, 'sample_weight')] = res['build_existing_model.sample_weight'].mean()
        summ[(level0, 'n_applied')] = len(p)
        summ[(level0, 'n_success')] = len(p[p['completed_status']=='Success'])
        summ[(level0, 'n_fail')] = len(p[p['completed_status']=='Fail'])
        summ[(level0, 'n_invalid')] = len(p[p['completed_status']=='Invalid'])
        summ[(level0, 'pct_success')] = round(summ[(level0,'n_success')]/summ[(level0,'n_applied')]*100,3)
        summ.columns = pd.MultiIndex.from_tuples(summ.columns)

        para = ['mean','std', 'min','50%','max']
        p = p[p['completed_status']=='Success'].reset_index(drop=True)

        ## (0) Upgrade cost
        summ = pd.concat([
            summ,
            p['upgrade_cost'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
        ], axis=1)

        ## (1) Baselines
        consum = pd.DataFrame(index=[0])
        level1 = 'baseline - mean'
        # 1.1. Annual Totals
        consum[(level1, 'total gas (therm)')] = round(
            res.loc[p.index, 'simulation_output_report.total_site_natural_gas_therm'].mean(), 
            3)
        consum[(level1, 'total elec (kwh)')] = round(
            res.loc[p.index, 'simulation_output_report.total_site_electricity_kwh'].mean(), 
            3)
        consum[(level1, 'total site (mbtu)')] = round(
            res.loc[p.index, 'simulation_output_report.total_site_energy_mbtu'].mean(), 
            3)

        # 1.2. Thermal Totals
        consum[(level1, 'thermal gas (therm)')] = round(
            res.loc[p.index, 'thermal_gas_therm'].mean(), 
            3)
        consum[(level1, 'thermal elec (kwh)')] = round(
            res.loc[p.index, 'thermal_elec_kwh'].mean(), 
            3)
        consum[(level1, 'thermal site (mbtu)')] = round(
            res.loc[p.index, 'thermal_site_mbtu'].mean(), 
            3)

        # 1.3. Thermal EUIs
        consum[(level1, 'thermal_gas_eui (therm/sqft)')] = round(
            res.loc[p.index, 'gas_eui_thermpersqft'].mean(), 
            3)
        consum[(level1, 'thermal_elec_eui (kwh/sqft)')] = round(
            res.loc[p.index, 'elec_eui_kwhpersqft'].mean(), 
            3)
        consum[(level1, 'thermal_site_eui (mbtu/sqft)')] = round(
            res.loc[p.index, 'site_eui_mbtupersqft'].mean(), 
            3)

        ## (2) Upgrades
        level2 = 'upgrade - mean'
        # 2.1. Annual Totals
        consum[(level2, 'gas (therm)')] = round(p['simulation_output_report.total_site_natural_gas_therm'].mean(), 3)
        consum[(level2, 'elec (kwh)')] = round(p['simulation_output_report.total_site_electricity_kwh'].mean(), 3)
        consum[(level2, 'site (mbtu)')] = round(p['simulation_output_report.total_site_energy_mbtu'].mean(), 3)

        # 2.2. EUIs
        consum[(level2, 'gas_eui (therm/sqft)')] = round(p['gas_eui_thermpersqft'].mean(), 3)
        consum[(level2, 'elec_eui (kwh/sqft)')] = round(p['elec_eui_kwhpersqft'].mean(), 3)
        consum[(level2, 'site_eui (mbtu/sqft)')] = round(p['site_eui_mbtupersqft'].mean(), 3)

        consum.columns = pd.MultiIndex.from_tuples(consum.columns)
        summ = pd.concat([summ, consum], axis=1)


        ## (3-1) Savings ##
        level3 = 'savings'
        # 3-1.1 Annual Totals
        summ = pd.concat([
            summ,
            p['ann_therm_gas_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
        ], axis=1)
        summ = pd.concat([
            summ,
            p['ann_kwh_elec_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
        ], axis=1)
        summ = pd.concat([
            summ,
            p['ann_mbtu_site_energy_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
        ], axis=1)

        summ = pd.concat([
            summ,
            p['pct_delta_gas_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
        ], axis=1)
        summ = pd.concat([
            summ,
            p['pct_delta_elec_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
        ], axis=1)
        summ = pd.concat([
            summ,
            p['pct_delta_site_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
        ], axis=1)



        summary_upgrades.append(summ)
    
summary_upgrades = pd.concat(summary_upgrades).reset_index(drop=True)
summary_upgrades


In [ ]:
### export UPGRADE summary
summary_upgrades.to_csv(outdir / f'upgrades_summary.csv', index=False)
print(f'UPGRADE summary table saved to new loc')